In [3]:
import Pkg;
Pkg.add("JSON"); Pkg.add("Ipopt"); Pkg.add("JuMP")
Pkg.add("PolyhedralRelaxations"); Pkg.add("SparseArrays")
Pkg.add("OpenBLAS32_jll")
using JuMP, Ipopt
using JSON
using PolyhedralRelaxations
using SparseArrays
using Plots

In [2]:
# Create a function to write output in a unique named folder
function mk_output_dir()
    i = 1
    while true
        dir_name = joinpath(@__DIR__, "./output", "run_$i")
        if !ispath(dir_name)
            mkpath(dir_name)
            return dir_name
        end
        i += 1
    end
end

mk_output_dir (generic function with 1 method)

## 8 Node Case 1

Without carbon offset

In [5]:
include("./src/NGHTwoSteadyOpt.jl")

# Choose which network to run
file = "./data/8-node/"
include("./data/8-node/CASE1.jl")

ss = initialize_optimizer(file, initial_guess_filename="");

type = "nlp_eq"
solver_options = Dict()

report,shadowPrice = run_optimizer(ss, type, solver_options);
println("done")

# SAVE OUTPUT
open("./output/8-node/CASE1_physical.json","w") do f
    JSON.print(f, report.sol)
end
open("./output/8-node/CASE1_shadowprice.json","w") do f
    JSON.print(f, shadowPrice)
end

This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:      143
Number of nonzeros in inequality constraint Jacobian.:       12
Number of nonzeros in Lagrangian Hessian.............:       82

Total number of variables............................:       40
                     variables with only lower bounds:       19
                variables with lower and upper bounds:       21
                     variables with only upper bounds:        0
Total number of equality constraints.................:       33
Total number of inequality constraints...............:        6
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        6

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  4.2908673e-01 4.28e-01 7.81e-01  -1.0 0.00e+00    -  0.00e+00 0.00e+00  

## 8 Node Case 2

With 0.055 carbon offset

In [6]:
include("./src/NGHTwoSteadyOpt.jl")

# Choose which network to run
file = "./data/8-node/"
include("./data/8-node/CASE2.jl")

ss = initialize_optimizer(file, initial_guess_filename="");

type = "nlp_eq"
solver_options = Dict()

report,shadowPrice = run_optimizer(ss, type, solver_options);
println("done")

# SAVE OUTPUT
open("./output/8-node/CASE2_physical.json","w") do f
    JSON.print(f, report.sol)
end
open("./output/8-node/CASE2_shadowprice.json","w") do f
    JSON.print(f, shadowPrice)
end

This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:      143
Number of nonzeros in inequality constraint Jacobian.:       12
Number of nonzeros in Lagrangian Hessian.............:       82

Total number of variables............................:       40
                     variables with only lower bounds:       19
                variables with lower and upper bounds:       21
                     variables with only upper bounds:        0
Total number of equality constraints.................:       33
Total number of inequality constraints...............:        6
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        6

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  4.2951742e-01 4.28e-01 7.81e-01  -1.0 0.00e+00    -  0.00e+00 0.00e+00  

## 40 Node Baseline

In [9]:
include("./src/NGHTwoSteadyOpt.jl")

# Choose which network to run
file = "./data/40-node-baseline/"
include("./data/40-node-baseline/CASE1.jl")

ss = initialize_optimizer(file, initial_guess_filename="");

type = "nlp_eq"
solver_options = Dict()

report,shadowPrice = run_optimizer(ss, type, solver_options);
println("done")

# SAVE OUTPUT
open("./output/40-node-baseline/CASE1_physical.json","w") do f
    JSON.print(f, report.sol)
end
open("./output/40-node-baseline/CASE1_shadowprice.json","w") do f
    JSON.print(f, shadowPrice)
end

This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:      851
Number of nonzeros in inequality constraint Jacobian.:       84
Number of nonzeros in Lagrangian Hessian.............:      489

Total number of variables............................:      208
                     variables with only lower bounds:      111
                variables with lower and upper bounds:       97
                     variables with only upper bounds:        0
Total number of equality constraints.................:      171
Total number of inequality constraints...............:       32
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:       32

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  6.9306641e+00 6.30e-01 3.21e+01  -1.0 0.00e+00    -  0.00e+00 0.00e+00  

In [10]:
summed_ng = 0
summed_h2 = 0
summed_total = 0
summed_energy = 0
summed_ci = 0
summed_dcp = 0
summed_λd = 0
for i = 1:length(report.sol[:total_ng])
    summed_ng += report.sol[:total_ng][i]
    summed_h2 += report.sol[:total_h2][i]
    summed_total += report.sol[:withdrawal_flows][i]
    summed_energy += report.sol[:withdrawal_energy][i]
    summed_ci += report.sol[:carbon_intensity][i]
    summed_dcp += report.sol[:node_decarbpremium][i]*report.sol[:withdrawal_energy][i]
    summed_λd += shadowPrice[:λd][i]*report.sol[:withdrawal_energy][i]
end
println("Total NG : ",summed_ng)
println("Total H2 : ", summed_h2)
println("Total Flow : ", summed_ng+summed_h2, "  ",summed_total)
println("Total Energy : ", summed_ng*44.2+summed_h2*141.8, "  ",summed_energy)
println("Average CI : ", report.sol[:total_co2][1]/summed_energy)
println("CO2 : ", report.sol[:total_co2][1])
println("Decarb premium : ", report.sol[:obj_fn_CEM ], "  ", summed_dcp, "   ", summed_λd)

Total NG : 726.7070850904207
Total H2 : 44.92613179053029
Total Flow : 771.6332168809511  771.6332168809507
Total Energy : 38490.97864889379  38490.978648893804
Average CI : 0.05191981482793796
CO2 : 1998.44448399668
Decarb premium : 21.79959230869799  21.799592308697985   21.797064481334655


## 40 Node Case 1

In [11]:
include("./src/NGHTwoSteadyOpt.jl")

# Choose which network to run
file = "./data/40-node-counter/"
include("./data/40-node-counter/CASE1.jl")

ss = initialize_optimizer(file, initial_guess_filename="");

type = "nlp_eq"
solver_options = Dict()

report,shadowPrice = run_optimizer(ss, type, solver_options);
println("done")

open("./output/40-node-counter/CASE1_physical.json","w") do f
    JSON.print(f, report.sol)
end
open("./output/40-node-counter/CASE1_shadowprice.json","w") do f
    JSON.print(f, shadowPrice)
end

This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:      866
Number of nonzeros in inequality constraint Jacobian.:       93
Number of nonzeros in Lagrangian Hessian.............:      507

Total number of variables............................:      211
                     variables with only lower bounds:      114
                variables with lower and upper bounds:       97
                     variables with only upper bounds:        0
Total number of equality constraints.................:      171
Total number of inequality constraints...............:       35
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:       35

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  7.8650132e+00 6.30e-01 3.21e+01  -1.0 0.00e+00    -  0.00e+00 0.00e+00  

In [12]:
summed_ng = 0
summed_h2 = 0
summed_total = 0
summed_energy = 0
summed_ci = 0
summed_dcp = 0
summed_λd = 0
for i = 1:length(report.sol[:total_ng])
    summed_ng += report.sol[:total_ng][i]
    summed_h2 += report.sol[:total_h2][i]
    summed_total += report.sol[:withdrawal_flows][i]
    summed_energy += report.sol[:withdrawal_energy][i]
    summed_ci += report.sol[:carbon_intensity][i]
    summed_dcp += report.sol[:node_decarbpremium][i]*report.sol[:withdrawal_energy][i]
    summed_λd += shadowPrice[:λd][i]*report.sol[:withdrawal_energy][i]
end
println("Total NG : ",summed_ng)
println("Total H2 : ", summed_h2)
println("Total Flow : ", summed_ng+summed_h2, "  ",summed_total)
println("Total Energy : ", summed_ng*44.2+summed_h2*141.8, "  ",summed_energy)
println("Average CI : ", report.sol[:total_co2][1]/summed_energy)
println("CO2 : ", report.sol[:total_co2][1])
println("Decarb premium : ", report.sol[:obj_fn_CEM ], "  ", summed_dcp, "   ", summed_λd)

Total NG : 687.563285066714
Total H2 : 76.39491875928509
Total Flow : 763.9582038259991  763.958203825999
Total Energy : 41223.096680015384  41223.09668001538
Average CI : 0.04586746717769282
CO2 : 1890.7990339334633
Decarb premium : 37.069251614481395  37.06925161448139   37.06641812884611


## 40 Node Case 2

In [13]:
include("./src/NGHTwoSteadyOpt.jl")

# Choose which network to run
file = "./data/40-node-counter/"
include("./data/40-node-counter/CASE2.jl")

ss = initialize_optimizer(file, initial_guess_filename="");

type = "nlp_eq"
solver_options = Dict()

report,shadowPrice = run_optimizer(ss, type, solver_options);
println("done")

open("./output/40-node-counter/CASE2_physical.json","w") do f
    JSON.print(f, report.sol)
end
open("./output/40-node-counter/CASE2_shadowprice.json","w") do f
    JSON.print(f, shadowPrice)
end

This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:      866
Number of nonzeros in inequality constraint Jacobian.:       93
Number of nonzeros in Lagrangian Hessian.............:      507

Total number of variables............................:      211
                     variables with only lower bounds:      114
                variables with lower and upper bounds:       97
                     variables with only upper bounds:        0
Total number of equality constraints.................:      171
Total number of inequality constraints...............:       35
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:       35

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  7.1769421e+00 6.30e-01 3.21e+01  -1.0 0.00e+00    -  0.00e+00 0.00e+00  

In [14]:
summed_ng = 0
summed_h2 = 0
summed_total = 0
summed_energy = 0
summed_ci = 0
summed_dcp = 0
summed_λd = 0
for i = 1:length(report.sol[:total_ng])
    summed_ng += report.sol[:total_ng][i]
    summed_h2 += report.sol[:total_h2][i]
    summed_total += report.sol[:withdrawal_flows][i]
    summed_energy += report.sol[:withdrawal_energy][i]
    summed_ci += report.sol[:carbon_intensity][i]
    summed_dcp += report.sol[:node_decarbpremium][i]*report.sol[:withdrawal_energy][i]
    summed_λd += shadowPrice[:λd][i]*report.sol[:withdrawal_energy][i]
end
println("Total NG : ",summed_ng)
println("Total H2 : ", summed_h2)
println("Total Flow : ", summed_ng+summed_h2, "  ",summed_total)
println("Total Energy : ", summed_ng*44.2+summed_h2*141.8, "  ",summed_energy)
println("Average CI : ", report.sol[:total_co2][1]/summed_energy)
println("CO2 : ", report.sol[:total_co2][1])
println("Decarb premium : ", report.sol[:obj_fn_CEM ], "  ", summed_dcp, "   ", summed_λd)

Total NG : 673.8045240133874
Total H2 : 74.86635210717243
Total Flow : 748.6708761205599  748.67087612056
Total Energy : 40398.20869018878  40398.20869018878
Average CI : 0.04586744068894398
CO2 : 1852.9624410368153
Decarb premium : 36.32754231290846  36.32754231290847   36.32471597287387


## 40 Node Case 3

In [15]:
include("./src/NGHTwoSteadyOpt.jl")

# Choose which network to run
file = "./data/40-node-counter/"
include("./data/40-node-counter/CASE3.jl")

ss = initialize_optimizer(file, initial_guess_filename="");

type = "nlp_eq"
solver_options = Dict()

report,shadowPrice = run_optimizer(ss, type, solver_options);
println("done")

open("./output/40-node-counter/CASE3_physical.json","w") do f
    JSON.print(f, report.sol)
end
open("./output/40-node-counter/CASE3_shadowprice.json","w") do f
    JSON.print(f, shadowPrice)
end

This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:      866
Number of nonzeros in inequality constraint Jacobian.:       93
Number of nonzeros in Lagrangian Hessian.............:      507

Total number of variables............................:      211
                     variables with only lower bounds:      114
                variables with lower and upper bounds:       97
                     variables with only upper bounds:        0
Total number of equality constraints.................:      171
Total number of inequality constraints...............:       35
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:       35

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  7.1864041e+00 6.30e-01 3.22e+01  -1.0 0.00e+00    -  0.00e+00 0.00e+00  

In [16]:
summed_ng = 0
summed_h2 = 0
summed_total = 0
summed_energy = 0
summed_ci = 0
summed_dcp = 0
summed_λd = 0
for i = 1:length(report.sol[:total_ng])
    summed_ng += report.sol[:total_ng][i]
    summed_h2 += report.sol[:total_h2][i]
    summed_total += report.sol[:withdrawal_flows][i]
    summed_energy += report.sol[:withdrawal_energy][i]
    summed_ci += report.sol[:carbon_intensity][i]
    summed_dcp += report.sol[:node_decarbpremium][i]*report.sol[:withdrawal_energy][i]
    summed_λd += shadowPrice[:λd][i]*report.sol[:withdrawal_energy][i]
end
println("Total NG : ",summed_ng)
println("Total H2 : ", summed_h2)
println("Total Flow : ", summed_ng+summed_h2, "  ",summed_total)
println("Total Energy : ", summed_ng*44.2+summed_h2*141.8, "  ",summed_energy)
println("Average CI : ", report.sol[:total_co2][1]/summed_energy)
println("CO2 : ", report.sol[:total_co2][1])
println("Decarb premium : ", report.sol[:obj_fn_CEM ], "  ", summed_dcp, "   ", summed_λd)

Total NG : 679.3330160531552
Total H2 : 75.48126596546152
Total Flow : 754.8142820186167  754.814282018617
Total Energy : 40729.76282345191  40729.76282345191
Average CI : 0.045867337903340334
CO2 : 1868.1657941461776
Decarb premium : 103.21849655658184  103.2184965565818   103.21566992066724
